In [5]:
import nltk  # Added this import
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

# Ensure stopwords are downloaded only once (not a coding error, but a good practice)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

STOPWORDS = stopwords.words('english')

# Third-party imports
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from tensorflow.keras import Sequential, layers

df = pd.read_csv('../data/Airline_review.csv')[['Review_Title','Review','Recommended']]
X = df['Review_Title'] + ' ' + df['Review']
y = df['Recommended'].map({'yes':1,'no':0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_sampled, _, y_train_sampled, _ = train_test_split(X_train, y_train, test_size=(len(X_train)-3000)/len(X_train), stratify=y_train, random_state=42)


In [12]:
text_cleaner = TextCleanerTransformer(stop_words=None, lemmatize=False)
X_train_clean = text_cleaner.transform(X_train)
X_train_clean_sampled = text_cleaner.transform(X_train_sampled)

# MLP

In [ ]:
# Define model
def build_mlp_model(num_layers=1, units=64, initializer=None, dropout_rate=0.2):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(20000,)))
    for _ in range(num_layers):
        model.add(layers.Dense(units, activation="relu", kernel_initializer=initializer))
        model.add(layers.Dropout(dropout_rate))
        units = max(8, units // 2)
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
    
# Setting parameters
k_best = SelectKBest(k=20000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='loss',
                                              min_delta=0.01,
                                              patience=5, 
                                              restore_best_weights=True,
                                              verbose=1,
                                              start_from_epoch=5)]

# Instantiate transformers
vectorizer = CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)
tf_idf = TfidfTransformer()
nlp_model_wrapper = KerasClassifier(build_fn=build_mlp_model,
                                random_state=42,
                                optimizer='adam',
                                loss='binary_crossentropy',
                                metrics=['accuracy'],
                                batch_size=64,
                                verbose=1,
                                callbacks=CALLBACKS,
                                shuffle=True,
                                epochs=20)
pipe = Pipeline([
    ("count", vectorizer),
    ('tf_idf', tf_idf),
    ('feature_selection', k_best),
    ('mlp', nlp_model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [8, 16, 32, 64],
    'mlp__model__initializer': [None,'he_normal'],
    'mlp__model__dropout_rate': [0.2, 0.3, 0.4, 0.5]}

gs_1 = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='balanced_accuracy',
                  cv=skf,
                  verbose=3,
                  error_score=0,
                  n_jobs= -1)

mlp_grid_search = gs_1.fit(X_train_clean, y_train)
summarize_mlp_grid_search_results(mlp_grid_search)

# RNN

In [14]:
# Define model
vocab=1000
length=100

class KerasTextVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_tokens=vocab, output_sequence_length=length):
        self.max_tokens = max_tokens
        self.output_sequence_length = output_sequence_length
        self.text_vectorization = tf.keras.layers.TextVectorization(
            max_tokens=self.max_tokens,
            output_sequence_length=self.output_sequence_length)

    def fit(self, X, y=None):
        self.text_vectorization.adapt(X)
        return self  # Return self to allow chaining

    def transform(self, X, y=None):
        return self.text_vectorization(X).numpy()  # Convert to numpy for sklearn compatibility

def add_rnn_layer(model, units, rnn_type='gru', bidirectional=False, return_sequences=False):
    LayerClass = layers.GRU if rnn_type == 'gru' else layers.LSTM
    layer = LayerClass(units, return_sequences=return_sequences)
    if bidirectional:
        layer = layers.Bidirectional(layer)
    model.add(layer)

def build_rnn_model(rnn_layers=1, dense_layers=1, recurrent_type='gru', bi_directional=False, dropout_rate=0.2, units=64, sequence_length=length, vocab_size=vocab):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=units, input_length=sequence_length))

    for i in range(rnn_layers):
        add_rnn_layer(model, units, rnn_type=recurrent_type, bidirectional=bi_directional, 
                      return_sequences=(i < rnn_layers - 1))
        model.add(layers.Dropout(dropout_rate))
        units = max(8, units // 2)

    for _ in range(dense_layers):
        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.Dropout(dropout_rate))
        units = max(8, units // 2)

    model.add(layers.Dense(1, activation='sigmoid'))
    return model

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='loss',
                                              min_delta=0.01,
                                              patience=2, 
                                              restore_best_weights=True,
                                              verbose=1)]

# Instantiate transformers
text_vectorizer = KerasTextVectorizer(max_tokens=vocab, output_sequence_length=length)
rnn_model_wrapper = KerasClassifier(build_fn=build_rnn_model,
                                    random_state=42,
                                    optimizer='adam',
                                    loss='binary_crossentropy',
                                    metrics=['accuracy'],
                                    batch_size=64,
                                    verbose=0,
                                    callbacks=CALLBACKS,
                                    shuffle=True,
                                    epochs=20)


pipe = Pipeline([
    ('text_vect', text_vectorizer),
    ('rnn', rnn_model_wrapper)
])

params = {
    'rnn__model__recurrent_type': ['gru','lstm'],  
    'rnn__model__units': [32,64],
    'rnn__model__bi_directional': [True, False],
    'rnn__model__rnn_layers': [1,2],
    'rnn__model__dense_layers': [1,2],
    'rnn__model__dropout_rate': [0.25]
}
gs_2 = GridSearchCV(estimator=pipe,
                    param_grid=params,
                    scoring='balanced_accuracy',
                    cv=skf,
                    verbose=4,
                    n_jobs= 1,
                    return_train_score=True)

rnn_grid_search = gs_2.fit(X_train_clean_sampled, y_train_sampled)
summarize_rnn_grid_search_results(rnn_grid_search)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Restoring model weights from the end of the best epoch: 11.
Epoch 13: early stopping
[CV 1/2] END rnn__model__bi_directional=True, rnn__model__dense_layers=1, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=gru, rnn__model__rnn_layers=1, rnn__model__units=32;, score=(train=0.983, test=0.822) total time=  15.4s


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Restoring model weights from the end of the best epoch: 11.
Epoch 13: early stopping
[CV 2/2] END rnn__model__bi_directional=True, rnn__model__dense_layers=1, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=gru, rnn__model__rnn_layers=1, rnn__model__units=32;, score=(train=0.981, test=0.827) total time=  15.2s


...

/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Restoring model weights from the end of the best epoch: 4.
Epoch 6: early stopping
[CV 2/2] END rnn__model__bi_directional=False, rnn__model__dense_layers=2, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=lstm, rnn__model__rnn_layers=2, rnn__model__units=64;, score=(train=0.770, test=0.738) total time=  18.4s


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Restoring model weights from the end of the best epoch: 11.
Epoch 13: early stopping


,train_score,test_score,time,units,bi_directional,recurrent_type,num_rnn_layers,num_dense_layers,dropout_rate
21,0.956994,0.854526,23,64,False,lstm,1,1,0.25
23,0.969257,0.848538,32,64,False,lstm,2,1,0.25
14,0.965835,0.848170,38,32,True,lstm,2,2,0.25
29,0.969451,0.847419,25,64,False,lstm,1,2,0.25
20,0.933583,0.838243,12,32,False,lstm,1,1,0.25
5,0.988463,0.836216,28,64,True,lstm,1,1,0.25
22,0.921656,0.832694,20,32,False,lstm,2,1,0.25
4,0.983528,0.830821,17,32,True,lstm,1,1,0.25
15,0.978276,0.826919,46,64,True,lstm,2,2,0.25
0,0.981978,0.824543,15,32,True,gru,1,1,0.25


In [15]:
# Define model
vocab=5000
length=400

class KerasTextVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_tokens=vocab, output_sequence_length=length):
        self.max_tokens = max_tokens
        self.output_sequence_length = output_sequence_length
        self.text_vectorization = tf.keras.layers.TextVectorization(
            max_tokens=self.max_tokens,
            output_sequence_length=self.output_sequence_length)

    def fit(self, X, y=None):
        self.text_vectorization.adapt(X)
        return self  # Return self to allow chaining

    def transform(self, X, y=None):
        return self.text_vectorization(X).numpy()  # Convert to numpy for sklearn compatibility

def add_rnn_layer(model, units, rnn_type='gru', bidirectional=False, return_sequences=False):
    LayerClass = layers.GRU if rnn_type == 'gru' else layers.LSTM
    layer = LayerClass(units, return_sequences=return_sequences)
    if bidirectional:
        layer = layers.Bidirectional(layer)
    model.add(layer)

def build_rnn_model(rnn_layers=1, dense_layers=1, recurrent_type='gru', bi_directional=False, dropout_rate=0.2, units=64, sequence_length=length, vocab_size=vocab):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=units, input_length=sequence_length))

    for i in range(rnn_layers):
        add_rnn_layer(model, units, rnn_type=recurrent_type, bidirectional=bi_directional, 
                      return_sequences=(i < rnn_layers - 1))
        model.add(layers.Dropout(dropout_rate))
        units = max(8, units // 2)

    for _ in range(dense_layers):
        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.Dropout(dropout_rate))
        units = max(8, units // 2)

    model.add(layers.Dense(1, activation='sigmoid'))
    return model

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='loss',
                                              min_delta=0.01,
                                              patience=2, 
                                              restore_best_weights=True,
                                              verbose=1)]

# Instantiate transformers
text_vectorizer = KerasTextVectorizer(max_tokens=vocab, output_sequence_length=length)
rnn_model_wrapper = KerasClassifier(build_fn=build_rnn_model,
                                    random_state=42,
                                    optimizer='adam',
                                    loss='binary_crossentropy',
                                    metrics=['accuracy'],
                                    batch_size=64,
                                    verbose=0,
                                    callbacks=CALLBACKS,
                                    shuffle=True,
                                    epochs=20)


pipe = Pipeline([
    ('text_vect', text_vectorizer),
    ('rnn', rnn_model_wrapper)
])

params = {
    'rnn__model__recurrent_type': ['gru','lstm'],  
    'rnn__model__units': [32,64,128,256],
    'rnn__model__bi_directional': [True, False],
    'rnn__model__rnn_layers': [1,2],
    'rnn__model__dense_layers': [1,2],
    'rnn__model__dropout_rate': [0, 0.25, 0.5]
}
gs_2 = GridSearchCV(estimator=pipe,
                    param_grid=params,
                    scoring='balanced_accuracy',
                    cv=skf,
                    verbose=4,
                    n_jobs= 1,
                    return_train_score=True)

rnn_grid_search = gs_2.fit(X_train_clean_sampled, y_train_sampled)
summarize_rnn_grid_search_results(rnn_grid_search)

Fitting 2 folds for each of 64 candidates, totalling 128 fits


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Restoring model weights from the end of the best epoch: 11.
Epoch 13: early stopping
[CV 1/2] END rnn__model__bi_directional=True, rnn__model__dense_layers=1, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=gru, rnn__model__rnn_layers=1, rnn__model__units=32;, score=(train=0.998, test=0.833) total time=  41.8s


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Restoring model weights from the end of the best epoch: 12.
Epoch 14: early stopping
[CV 2/2] END rnn__model__bi_directional=True, rnn__model__dense_layers=1, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=gru, rnn__model__rnn_layers=1, rnn__model__units=32;, score=(train=1.000, test=0.800) total time=  44.3s


...

Restoring model weights from the end of the best epoch: 6.
Epoch 8: early stopping
[CV 2/2] END rnn__model__bi_directional=True, rnn__model__dense_layers=1, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=gru, rnn__model__rnn_layers=1, rnn__model__units=128;, score=(train=1.000, test=0.831) total time= 2.5min


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Restoring model weights from the end of the best epoch: 7.
Epoch 9: early stopping
[CV 1/2] END rnn__model__bi_directional=True, rnn__model__dense_layers=1, rnn__model__dropout_rate=0.25, rnn__model__recurrent_type=gru, rnn__model__rnn_layers=1, rnn__model__units=256;, score=(train=1.000, test=0.823) total time= 5.4min


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


KeyboardInterrupt: 

In [13]:
summarize_rnn_grid_search_results(rnn_grid_search)

,train_score,test_score,time,units,bi_directional,recurrent_type,num_rnn_layers,num_dense_layers,dropout_rate
1,0.521731,0.519046,7,32,True,lstm,1,1,0.25
2,0.500000,0.500000,4,32,False,gru,1,1,0.25
3,0.500000,0.500000,5,32,False,lstm,1,1,0.25
0,0.500000,0.500000,7,32,True,gru,1,1,0.25
